<a href="https://colab.research.google.com/github/raki-rankawat/stm32/blob/main/VWW_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Trained Model (VWW - same style as CIFAR-10)

import os
import time
import tarfile
import random
import shutil
from pathlib import Path
from urllib.request import urlretrieve

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# ----------------------------
# Auto Download + Prepare VWW (10k subset)
# ----------------------------
# VWW preprocessed dataset (96x96) download URL
vww_url = "https://www.silabs.com/public/files/github/machine_learning/benchmarks/datasets/vw_coco2014_96.tar.gz"

base_dir = Path("/content/vww_work")
archive_path = base_dir / "vw_coco2014_96.tar.gz"
extract_dir = base_dir / "extracted"
subset_dir = base_dir / "vww_10k"

# Subset config: 5k person + 5k non_person
n_per_class = 5000
val_ratio = 0.20

random.seed(41)
torch.manual_seed(41)

def download_vww():
    base_dir.mkdir(parents=True, exist_ok=True)

    if archive_path.exists() and archive_path.stat().st_size > 0:
        print("✅ VWW archive already downloaded")
        return

    print("⬇️ Downloading VWW archive...")
    urlretrieve(vww_url, archive_path)
    print("✅ Download complete:", archive_path)

def extract_vww():
    extract_dir.mkdir(parents=True, exist_ok=True)

    if any(extract_dir.iterdir()):
        print("✅ VWW already extracted")
        return

    print("📦 Extracting VWW archive...")
    with tarfile.open(archive_path, "r:gz") as tar:
        tar.extractall(extract_dir)
    print("✅ Extraction complete:", extract_dir)

def find_vww_root():
    # Find folder that contains BOTH person/ and non_person/
    for p in extract_dir.rglob("person"):
        if p.is_dir() and (p.parent / "non_person").is_dir():
            return p.parent
    raise RuntimeError("❌ Could not find 'person' and 'non_person' directories under extracted dataset")

def list_images(folder):
    exts = {".jpg", ".jpeg", ".png"}
    return [p for p in folder.rglob("*") if p.is_file() and p.suffix.lower() in exts]

def make_vww_subset(src_root):
    # Skip if subset already exists
    if (subset_dir / "train" / "person").is_dir() and (subset_dir / "val" / "non_person").is_dir():
        print("✅ VWW 10k subset already exists:", subset_dir)
        return

    for split in ["train", "val"]:
        for c in ["person", "non_person"]:
            (subset_dir / split / c).mkdir(parents=True, exist_ok=True)

    person_imgs = list_images(src_root / "person")
    nonperson_imgs = list_images(src_root / "non_person")

    if len(person_imgs) < n_per_class or len(nonperson_imgs) < n_per_class:
        raise ValueError(
            f"❌ Not enough images:\n"
            f"person: {len(person_imgs)} (need {n_per_class})\n"
            f"non_person: {len(nonperson_imgs)} (need {n_per_class})"
        )

    random.shuffle(person_imgs)
    random.shuffle(nonperson_imgs)

    person_sel = person_imgs[:n_per_class]
    nonperson_sel = nonperson_imgs[:n_per_class]

    def split_list(lst, val_ratio):
        n_val = int(len(lst) * val_ratio)
        return lst[n_val:], lst[:n_val]  # train, val

    p_train, p_val = split_list(person_sel, val_ratio)
    n_train, n_val = split_list(nonperson_sel, val_ratio)

    def copy_files(files, dst_dir):
        for f in files:
            dst = dst_dir / f.name
            # avoid rare collisions
            if dst.exists():
                dst = dst_dir / (f"{f.parent.name}_{f.name}")
            shutil.copy2(f, dst)

    print("🧩 Creating VWW 10k subset...")
    copy_files(p_train, subset_dir / "train" / "person")
    copy_files(p_val,   subset_dir / "val"   / "person")
    copy_files(n_train, subset_dir / "train" / "non_person")
    copy_files(n_val,   subset_dir / "val"   / "non_person")
    print("✅ VWW subset created at:", subset_dir)

download_vww()
extract_vww()
vww_root = find_vww_root()
print("✅ Found VWW root:", vww_root)
make_vww_subset(vww_root)

✅ VWW archive already downloaded
✅ VWW already extracted
✅ Found VWW root: /content/vww_work/extracted/vw_coco2014_96
✅ VWW 10k subset already exists: /content/vww_work/vww_10k


In [4]:
# ----------------------------
# Data Loaders (same style)
# ----------------------------
batch_size = 64
img_size = 96

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(img_size, scale=(0.6, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

train_data = datasets.ImageFolder(root=str(subset_dir / "train"), transform=train_transform)
test_data  = datasets.ImageFolder(root=str(subset_dir / "val"),   transform=test_transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=batch_size, shuffle=False)

print("Class mapping:", train_data.class_to_idx)

Class mapping: {'non_person': 0, 'person': 1}


In [5]:
# ----------------------------
# CNN Model (same style)
# ----------------------------
class VWWConvNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(128)

        # For 96x96 with 4 pools: 96->48->24->12->6
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 2)   # VWW: person vs non_person

        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2) # 96 -> 48

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2) # 48 -> 24

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2) # 24 -> 12

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool2d(x, 2) # 12 -> 6

        x = x.view(x.size(0), -1) # Flatten

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x

In [6]:
# ----------------------------
# Random Seeds and Model Instance (same style)
# ----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(41)
model = VWWConvNet().to(device)

In [7]:
# ----------------------------
# Loss & Optimizer (same style)
# ----------------------------
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=1e-4)

In [8]:
# ----------------------------
# Training (same style)
# ----------------------------
def train(model, loader, criterion, optimizer, device):
    model.train() # training mode

    correct = 0
    total = 0
    running_loss = 0

    for X, y in loader:
        X = X.to(device)
        y = y.to(device)

        # Forward
        outputs = model(X)

        # Loss
        loss = criterion(outputs, y)

        # Backward
        optimizer.zero_grad(set_to_none = True)
        loss.backward()
        optimizer.step()

        # Statistics
        batch_size = y.size(0)
        running_loss += loss.item() * batch_size
        preds = outputs.argmax(dim = 1)
        correct += (preds == y).sum().item()
        total += batch_size

    avg_loss = running_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

In [9]:
# ----------------------------
# Testing (same style)
# ----------------------------
def test(model, loader, criterion, device):
    model.eval() # testing mode

    correct = 0
    total = 0
    running_loss = 0

    with torch.no_grad():
        for X, y in loader:
            X = X.to(device)
            y = y.to(device)

            # Forward
            outputs = model(X)

            # Loss
            loss = criterion(outputs, y)

            # Statistics
            batch_size = y.size(0)
            running_loss += loss.item() * batch_size
            preds = outputs.argmax(dim = 1)
            correct += (preds == y).sum().item()
            total += batch_size

    avg_loss = running_loss / total
    accuracy = correct / total

    return avg_loss, accuracy

In [10]:
# ----------------------------
# Run (same style print)
# ----------------------------
epochs = 30
start_time = time.time()
best_acc = 0
best_epoch = 1

for epoch in range(1, epochs + 1):
    train_loss, train_acc = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_acc = test(model, test_loader, criterion, device)

    print(
        f"Epoch: {epoch}/{epochs} | "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc * 100:.2f}% | "
        f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc * 100:.2f}%"
    )

    if test_acc > best_acc:
        best_acc = test_acc
        best_epoch = epoch

print(f"Training Time: {(time.time() - start_time) / 60} minutes!")

if best_acc > 0:
  torch.save(model.state_dict(), "/content/drive/My Drive/Colab Notebooks/stm_vww_best.pth")
  print("✅ Model saved as stm_vww_model.pth for epoch", best_epoch)

Epoch: 1/30 | Train Loss: 0.6811 | Train Acc: 60.25% | Test Loss: 0.6614 | Test Acc: 58.90%
Epoch: 2/30 | Train Loss: 0.6284 | Train Acc: 65.01% | Test Loss: 0.6052 | Test Acc: 67.60%
Epoch: 3/30 | Train Loss: 0.6065 | Train Acc: 68.23% | Test Loss: 0.5901 | Test Acc: 68.60%
Epoch: 4/30 | Train Loss: 0.5906 | Train Acc: 69.30% | Test Loss: 0.6126 | Test Acc: 65.95%
Epoch: 5/30 | Train Loss: 0.5813 | Train Acc: 70.34% | Test Loss: 0.7081 | Test Acc: 61.35%
Epoch: 6/30 | Train Loss: 0.5719 | Train Acc: 71.21% | Test Loss: 0.5887 | Test Acc: 68.80%
Epoch: 7/30 | Train Loss: 0.5632 | Train Acc: 71.58% | Test Loss: 0.5499 | Test Acc: 71.25%
Epoch: 8/30 | Train Loss: 0.5534 | Train Acc: 72.51% | Test Loss: 0.5593 | Test Acc: 70.80%
Epoch: 9/30 | Train Loss: 0.5468 | Train Acc: 72.84% | Test Loss: 0.5993 | Test Acc: 68.75%
Epoch: 10/30 | Train Loss: 0.5389 | Train Acc: 73.70% | Test Loss: 0.5457 | Test Acc: 72.60%
Epoch: 11/30 | Train Loss: 0.5313 | Train Acc: 73.96% | Test Loss: 0.5982 | Tes